In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash

In [ ]:
#loading data
crs_data = pd.read_csv("crs_data.csv")

In [ ]:
#understanding variables
#crs_data.head()
#crs_data.info()

,donor_code,donor_name,recipient_code,recipient_name,sector_code,sector_name,measure_code,measure_name,channel_code,channel_name,...,drr,nutrition,disability,biodiversity,climate_mitigation,climate_adaptation,desertification,amount_tied,amount_untied,nature_of_submission
0,1628.0,Wellcome Trust,665,Pakistan,12182,Medical research,30,Private Development Finance,50000,"Teaching institutions, research institutes or ...",...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,8.0
1,1628.0,Wellcome Trust,665,Pakistan,12182,Medical research,30,Private Development Finance,50000,"Teaching institutions, research institutes or ...",...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,8.0
2,1628.0,Wellcome Trust,665,Pakistan,12182,Medical research,30,Private Development Finance,50000,"Teaching institutions, research institutes or ...",...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,8.0
3,1628.0,Wellcome Trust,665,Pakistan,12182,Medical research,30,Private Development Finance,50000,"Teaching institutions, research institutes or ...",...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,8.0
4,1628.0,Wellcome Trust,665,Pakistan,12220,Basic health care,30,Private Development Finance,50000,"Teaching institutions, research institutes or ...",...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,8.0


In [ ]:
#figuring out what the unique donor names are, to categorise
#unique_donor_name = crs_data["donor_name"].unique()
#print(unique_donor_name)
#unique_donors.to_csv('unique_donors_250517.csv', index=False)

['Wellcome Trust' 'IKEA Foundation' 'William and Flora Hewlett Foundation'
 'Jacobs Foundation' 'German Postcode Lottery' 'Open Society Foundations'
 'UBS Optimus Foundation' 'Susan T. Buffett Foundation'
 'World Diabetes Foundation' 'Gates Foundation' 'Rockefeller Foundation'
 'Laudes Foundation' 'H&M Foundation'
 'David and Lucile Packard Foundation' 'LEGO Foundation'
 "Children's Investment Fund Foundation" 'Conrad N. Hilton Foundation'
 'Ford Foundation' 'Good Ventures Foundation' 'Citi Foundation'
 'Bloomberg Family Foundation'
 'OPEC Fund for International Development [OPEC Fund]' 'Arcadia Fund'
 'Global Fund' 'Green Climate Fund [GCF]'
 'International Centre for Genetic Engineering and Biotechnology [ICGEB]'
 'Private Infrastructure Development Group'
 'Global Alliance for Vaccines and Immunization [GAVI]'
 'Global Environment Facility [GEF]' 'Adaptation Fund'
 'International Development Association [IDA]'
 'Nordic Development Fund [NDF]'
 'International Bank for Reconstruction 

In [ ]:
#Grouping donors into mutually exclusive categories - G7, BRICS, UN, World Bank, Other - etc

g7 = ["United Kingdom", 
      "United States", 
      "France", 
      "Germany", 
      "Italy", 
      "Canada", 
      "Japan"]

brics = ["Brazil", 
         "Russia", 
         "India",
         "China", 
         "South Africa"] # no BRIC countries currently in dataset

eu_institutions = ["EU Institutions"]

foundation = ["Wellcome Trust",
            "IKEA Foundation",
            "William and Flora Hewlett Foundation",
            "Jacobs Foundation",
            "German Postcode Lottery",
            "Open Society Foundations",
            "UBS Optimus Foundation",
            "Susan T. Buffett Foundation",
            "World Diabetes Foundation",
            "Gates Foundation",
            "Rockefeller Foundation",
            "Laudes Foundation",
            "H&M Foundation",
            "David and Lucile Packard Foundation",
            "LEGO Foundation",
            "Children's Investment Fund Foundation",
            "Conrad N. Hilton Foundation",
            "Ford Foundation",
            "Good Ventures Foundation",
            "Citi Foundation",
            "Bloomberg Family Foundation",
            "Arcadia Fund"
            ]

un_agency = ["International Centre for Genetic Engineering and Biotechnology [ICGEB]",
            "WFP",
            "United Nations Industrial Development Organization [UNIDO]",
            "UNICEF",
            "UNFPA",
            "UNHCR",
            "UNDP",
            "UNAIDS",
            "UN Women",
            "International Labour Organisation [ILO]",
            "Joint Sustainable Development Goals Fund [Joint SDG Fund]",
            "UN Development Coordination Office",
            "Food and Agriculture Organisation [FAO]",
            "International Atomic Energy Agency [IAEA]",
            "Central Emergency Response Fund [CERF]",
            "IFAD",
            "COVID-19 Response and Recovery Multi-Partner Trust Fund [UN COVID-19 MPTF]"
            ]

world_bank = ["International Development Association [IDA]",
              "International Bank for Reconstruction and Development [IBRD]"]

other_dfi_ifi_bank = ["OPEC Fund for International Development [OPEC Fund]",
                "Global Fund",
                "Green Climate Fund [GCF]"
                "Private Infrastructure Development Group",
                "Global Environment Facility [GEF]",
                "Adaptation Fund",
                "Nordic Development Fund [NDF]",
                "Islamic Development Bank [IsDB]",
                "Asian Infrastructure Investment Bank [AIIB]",
                "Asian Development Bank [AsDB]",
                "IMF Concessional Trust Funds"
                ]

other_igo_ngo_ppp = ["World Health Organisation [WHO]",
                "World Trade Organisation",
                "WTO - International Trade Centre [ITC]",
                "WHO-Strategic Preparedness and Response Plan [SPRP]",
                "Global Alliance for Vaccines and Immunization [GAVI]"
                ]

other_country = ["Australia",
            "Austria",
            "Azerbaijan",
            "Belgium",
            "Bulgaria",
            "Croatia",
            "Czechia",
            "Denmark",
            "Estonia",
            "Finland",
            "Greece",
            "Hungary",
            "Iceland",
            "Ireland",
            "Israel",
            "Kazakhstan",
            "Korea",
            "Kuwait",
            "Latvia",
            "Liechtenstein",
            "Lithuania",
            "Luxembourg",
            "Malta",
            "Netherlands",
            "New Zealand",
            "Norway",
            "Poland",
            "Portugal",
            "Qatar",
            "Romania",
            "Saudi Arabia",
            "Slovak Republic",
            "Slovenia",
            "Spain",
            "Sweden",
            "Switzerland",
            "Türkiye",
            "Thailand",
            "United Arab Emirates"
]



In [ ]:
# define function to apply donor grouping
def donor_group_function (donor):
    if donor in g7:
        return "G7"
    elif donor in brics:
        return "BRICS"
    elif donor in eu_institutions:
        return "EU Institutions"
    elif donor in foundation:
        return "Foundation"
    elif donor in un_agency:
        return "UN Agency"
    elif donor in world_bank:
        return "World Bank"
    elif donor in other_country:
        return "Other - Country"
    elif donor in other_dfi_ifi_bank:
        return "Other - DFI, IFI, Bank"
    else:
        return "Other - IGO, NGO, PPP"
    
#apply function crs_data to a new variable called 'donor group'
crs_data["donor_group"] = crs_data["donor_name"].apply(donor_group_function)

In [ ]:
#grouping dataset, calculating sum and rounding to billion - for charts
crs_grouped = crs_data.groupby(['year','donor_group'], as_index=False)['value'].sum()
crs_grouped["value_bn"] = crs_grouped["value"] / 1000
crs_grouped["value_bn_rounded"] = crs_grouped["value_bn"].round(2)

crs_grouped.head()

,year,donor_group,value,value_bn,value_bn_rounded
0,2002,G7,4361.032566,4.361033,4.36
1,2002,Other - Country,131.898886,0.131899,0.13
2,2002,"Other - DFI, IFI, Bank",859.946212,0.859946,0.86
3,2002,UN Agency,39.043041,0.039043,0.04
4,2002,World Bank,2443.378006,2.443378,2.44


#### EXPLORING VISUALISATIONS WITH PLOTLY EXPRESS

In [ ]:
fig = px.bar(
    crs_grouped,
    x="year",
    y="value_bn_rounded",
    color="donor_group",   # This separates the stacks by donor
    text_auto=True,       # Optional: shows values on bars
    title="ODA flows to Pakistan by year and donor group",
    labels={
        "year": "Year",
        "value_bn_rounded": "ODA disbursement ($billion)",
        "donor_group": "Donor Group"
    },
    color_discrete_sequence=px.colors.qualitative.Safe  # Optional: choose nice colors
)

# Customize the layout a bit
fig.update_layout(
    barmode="stack",  # Stack the bars
    title_font=dict(family="Arial", size=16),
    xaxis_title_font=dict(family="Arial", size=12),
    yaxis_title_font=dict(family="Arial", size=12),
    legend_title="Donor",
    template="plotly_white"
)

fig.show()

### definitely need to recategorise donor names, grouping roughly as follows:
# multilaterals: ADB, WB(IDA & IBRD), UN agencies, EU institution
# bilterals: USA, UK, France, Germany, Australia, Japan, Saudi, 'other'